In [ ]:
from mbarq.counter import BarcodeCounter
from pathlib import Path

In [ ]:
root = Path("/nfs/cds-peta/exports/biol_micro_cds_gr_sunagawa/scratch/Projects_NCCR/ref/mbarq_test_data/dnaid1315")

In [ ]:
sequencing_file = root/"test_data/dnaid1315_10.fasta.gz"
barcode_structure = "B17N13GTGTATAAGAGACAG"
mapping_file = root/"ref/library_11_1.annotated.csv"
edit_distance = 2

In [ ]:
bc = BarcodeCounter(sequencing_file, barcode_structure, 'test', mapping_file)

In [ ]:
bc._extract_barcodes()

In [ ]:
len(list(bc.barcode_counter.keys()))

In [ ]:
len(set(bc.barcode_counter.keys()))

In [ ]:
bc._merge_similar()

In [ ]:
len(list(bc.barcode_counter.keys()))

In [ ]:
len(set(bc.barcode_counter.keys()))

In [ ]:
bc._annotate_barcodes()

In [ ]:
def _annotate_barcodes(self) -> None:
    """
    If annotation is available, annotate the counted barcodes
    :return: None
    """
    assert self.barcode_counter is not None
    self.logger.info(f'Barcodes with edit distance of less than {self.edit_distance} '
                     f'have been merged: {self.merged}')
    cnts_df = (pd.DataFrame.from_dict(self.barcode_counter, orient='index')
               .reset_index())
    # Important to keep these column names, used for merging
    cnts_df.columns = ['barcode', 'barcode_count']
    cnts_df = cnts_df[cnts_df['barcode_count'] > 1]
    if cnts_df.empAty:
        self.logger.error('No barcodes with counts > 1 found')
        sys.exit(1)
    self.logger.info(f'Number of unique barcodes to annotate: {cnts_df.barcode.nunique()}')
    if self.bc_annotations.empty:
        self.logger.info('No mapping file provided, skipping the annotation')
        self.annotated_cnts = cnts_df
    else:
        self.logger.info(f'Annotating barcodes')
        self.logger.info(f'Columns found in the mapping/annotations file: '
                         f'{", ".join(self.bc_annotations.columns)}')
        self.annotated_cnts = cnts_df.merge(self.bc_annotations, how='left', on='barcode').drop_duplicates()
        filter_col = self.bc_annotations.columns[1]
        with_ids = self.annotated_cnts[self.annotated_cnts[filter_col].notnull()]
        self.logger.info(f'Number of annotated barcodes: {with_ids.shape[0]}')
        no_ids = self.annotated_cnts[self.annotated_cnts[filter_col].isna()]
        self.logger.info(f'Number of unannotated barcodes:{no_ids.shape[0]}')

In [ ]:
bc.annotated_cnts

In [ ]:
bc.annotated_cnts.barcode.nunique()

In [ ]:
df = bc.annotated_cnts.copy()

In [ ]:
df2 = df.groupby('barcode').barcode_count.count().reset_index()
df2[df2.barcode_count > 1]

In [ ]:
df[df.barcode == 'TCCACTGGAAGGAGAGC']